# 1) Break-beat - Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [14]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = 'jBeTEa5Uib8zQYrliQ0dTol1mgJtAZz5ui3Mqb01'  # Please replace by your own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME_BREAK = 'dataframe-break.csv'  # File where we'll store the metadata of our sounds collection
DATAFRAME_FILENAME_PIANO = 'dataframe-piano.csv'  # File where we'll store the metadata of our sounds collection

FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [15]:
# Define some util functions
def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

## 1.1. Break Beat Source Gathering

In [16]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    # idea is for the zaps to be lower drum kicks
    # idea for snare to be glass smash
    {
        'query': 'bass',
        'filter': 'duration:[0 TO 1]',
        'num_results': 20, 
    },
    {
        'query': 'sub',
        'filter': 'duration:[0 TO 2]',
        'num_results': 20, 
    }, 
    {
        'query': '303',
        'filter': 'duration:[0 TO 2]',
        'num_results': 20, 
    },
    {
        'query': 'clap',
        'filter': 'duration:[0 TO 2]',
        'num_results': 20, 
    }
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME_BREAK)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME_BREAK))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 80 entries! dataframe-break.csv


,freesound_id,license,name,path,tags,username
0,75229,http://creativecommons.org/publicdomain/zero/1.0/,BASS 0102.wav,files/75229_377011-hq.ogg,[bass],zgump
1,83251,http://creativecommons.org/publicdomain/zero/1.0/,BASS 0207.wav,files/83251_377011-hq.ogg,[bass],zgump
2,21585,http://creativecommons.org/licenses/by-nc/3.0/,sub bass stab 1.wav,files/21585_45941-hq.ogg,"[bass, noise, stab, sub, synth]",djgriffin
3,288878,http://creativecommons.org/licenses/by/3.0/,ElectroBass4.wav,files/288878_1471115-hq.ogg,"[hollow, short, electronic, bass, round]",gellski
4,4445,http://creativecommons.org/licenses/by/3.0/,BASS01.aif,files/4445_4948-hq.ogg,"[bass, free, multisample, sample, sound, synth]",NoiseCollector
5,144150,http://creativecommons.org/publicdomain/zero/1.0/,DK´s Hardcore Bass 02.wav,files/144150_1779635-hq.ogg,"[Hardcore, hard, bass, Kick]",e30029
6,25594,http://creativecommons.org/publicdomain/zero/1.0/,Love To Bass It.aif,files/25594_48671-hq.ogg,"[analogue, bass, fat, phat, square, sub, synth]",Walter_Odington
7,115525,http://creativecommons.org/licenses/by/3.0/,Bass Stab 11.wav,files/115525_2005415-hq.ogg,"[bass, dnb, dubstep, hit, stab, sub, wobble]",Ongitak
8,331261,http://creativecommons.org/publicdomain/zero/1.0/,modular hits#09.41.aif,files/331261_2050105-hq.ogg,"[modular, bass, synth, fm]",modularsamples
9,13893,http://creativecommons.org/publicdomain/zero/1.0/,41.wav,files/13893_33634-hq.ogg,"[bass, big, boom, kick, low, metal, object, th...",adcbicycle


## 1.2. Piano Source Gathering

In [79]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    # idea is for the zaps to be lower drum kicks
    # idea for snare to be glass smash
    {
        'query': 'JX-8P',
        'filter': None,
        'num_results': 800, 
    }
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME_PIANO)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME_PIANO))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 150 entries! dataframe-piano.csv


,freesound_id,license,name,path,tags,username
0,20358,http://creativecommons.org/licenses/sampling+/...,04A1.wav,files/20358_28848-hq.ogg,"[analog, jx-8p, lead, synth]",insky60
1,309756,http://creativecommons.org/publicdomain/zero/1.0/,Roland JX-8P - Synthetic - D4 (62D3-P5UV.aif),files/309756_2050105-hq.ogg,"[multisample, single-note, synthesizer, Roland...",modularsamples
2,309923,http://creativecommons.org/publicdomain/zero/1.0/,Roland JX-8P - Vulgar - A#2 (46A#1-P4U4.aif),files/309923_2050105-hq.ogg,"[multisample, single-note, synthesizer, Roland...",modularsamples
3,135144,http://creativecommons.org/publicdomain/zero/1.0/,Logan's Run,files/135144_615581-hq.ogg,"[sci-fi, fx, ambient, space, analog, Roland, J...",drriquet
4,135166,http://creativecommons.org/publicdomain/zero/1.0/,The Hitchhiker.wav,files/135166_615581-hq.ogg,"[Sci-Fi, UFO, FX, Weird, Analog, JX8P, Roland,...",drriquet
5,309372,http://creativecommons.org/publicdomain/zero/1.0/,Roland JX-8P - Sensitive Sync - C5 (72C4-P23P....,files/309372_2050105-hq.ogg,"[multisample, single-note, synthesizer, Roland...",modularsamples
6,308715,http://creativecommons.org/publicdomain/zero/1.0/,Roland JX-8P - Sample & Hold it - C#6 (85C#5-8...,files/308715_2050105-hq.ogg,"[multisample, single-note, synthesizer, Roland...",modularsamples
7,308242,http://creativecommons.org/publicdomain/zero/1.0/,Roland JX-8P - Mod Pad - E#2 (41F1-K8G8.aif),files/308242_2050105-hq.ogg,"[multisample, single-note, synthesizer, Roland...",modularsamples
8,307883,http://creativecommons.org/publicdomain/zero/1.0/,Roland JX-8P - Buzzy Lead - D4 (62D3-714P.aif),files/307883_2050105-hq.ogg,"[multisample, single-note, synthesizer, Roland...",modularsamples
9,309804,http://creativecommons.org/publicdomain/zero/1.0/,Roland JX-8P - Thin Brassy - C#3 (49C#2-55P2.aif),files/309804_2050105-hq.ogg,"[multisample, single-note, synthesizer, Roland...",modularsamples
